In [27]:
import pandas as pd
import os
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from time import sleep

# Fit regression model
import numpy as np
from sklearn.linear_model import LinearRegression

import warnings
warnings.filterwarnings("ignore")

In [28]:
def write_image(fig, fname, show=False):
    if show:
        fig.show()

    # create random figure to load math js
    fig2 = px.scatter(x=[0, 1], y=[0, 1])
    fig2.write_image(fname)
    sleep(1)

    fig.write_image(fname)

#### for each directory, consider last step and get the following metrics:
- avg_baseline0_sum_tenant_costs
- avg_baseline1_sum_tenant_costs
- avg_baseline0_accumulated_cost 
- avg_baseline1_accumulated_cost
- avg_baseline0_no_market_accumulated_cost
- avg_baseline1_no_market_accumulated_cost
- avg_baseline0_absolute_accumulated_saving
- avg_baseline1_absolute_accumulated_saving
- avg_baseline0_percentage_accumulated_saving
- avg_baseline1_percentage_accumulated_saving


In [29]:
dirs = [item for item in os.listdir('logs/YN_YY') if "NRECS" in item]

def get_summary_df(directory):
    baseline0_rows = []
    baseline1_rows = []

    for file in os.listdir(os.path.join("logs/YN_YY", directory)):
        if file.endswith(".csv"):
            filepath = os.path.join("logs/YN_YY", directory, file)
            df_yn_yy = pd.read_csv(filepath)
            last_row = df_yn_yy.iloc[-1]

            if "BASELINE0" in file:
                baseline0_rows.append(last_row)
            elif "BASELINE1" in file:
                baseline1_rows.append(last_row)

    # Convert lists of rows to DataFrames
    if baseline0_rows:
        baseline0_df_yn_yy = pd.DataFrame(baseline0_rows)
    else:
        baseline0_df_yn_yy = pd.DataFrame()

    if baseline1_rows:
        baseline1_df_yn_yy = pd.DataFrame(baseline1_rows)
    else:
        baseline1_df_yn_yy = pd.DataFrame()

    return baseline0_df_yn_yy, baseline1_df_yn_yy

summary_data = []
for directory in dirs:
    baseline0_df_yn_yy, baseline1_df_yn_yy = get_summary_df(directory)
    n_recs = int(directory.split("_")[0].replace("NRECS", ""))
    n_tenants = int(directory.split("_")[1].replace("NTEN", ""))

    # Calculate averages for specified metrics
    result = {
        ("RECs", ""): n_recs,
        ("Tenants", ""): n_tenants,
        ("Heuristics-based Agent", "YN | Avg. Cost ($)"): baseline0_df_yn_yy["no_market_accumulated_cost"].mean() if not baseline0_df_yn_yy.empty else None,
        ("Heuristics-based Agent", "YY | Avg. Cost ($)"): baseline0_df_yn_yy["accumulated_cost"].mean() if not baseline0_df_yn_yy.empty else None,
        ("Heuristics-based Agent", "Saving ($)"): baseline0_df_yn_yy["absolute_accumulated_saving"].mean() if not baseline0_df_yn_yy.empty else None,
        ("Heuristics-based Agent", "%"): baseline0_df_yn_yy["percentage_accumulated_saving"].mean() if not baseline0_df_yn_yy.empty else None,
        ("Best Agent", "YN | Avg. Cost ($)"): baseline1_df_yn_yy["no_market_accumulated_cost"].mean() if not baseline1_df_yn_yy.empty else None,
        ("Best Agent", "YY | Avg. Cost ($)"): baseline1_df_yn_yy["accumulated_cost"].mean() if not baseline1_df_yn_yy.empty else None,
        ("Best Agent", "Saving (%)"): baseline1_df_yn_yy["absolute_accumulated_saving"].mean() if not baseline1_df_yn_yy.empty else None,
        ("Best Agent", "%"): baseline1_df_yn_yy["percentage_accumulated_saving"].mean() if not baseline1_df_yn_yy.empty else None,
    }

    summary_data.append(result)

# Create the DataFrame with MultiIndex columns
summary_df_yn_yy = pd.DataFrame(summary_data)
summary_df_yn_yy.columns = pd.MultiIndex.from_tuples(summary_df_yn_yy.columns)

summary_df_yn_yy = summary_df_yn_yy[summary_df_yn_yy["Tenants"] != 1]

summary_df_yn_yy.sort_values(by=["RECs", "Tenants"], ascending=[True, True])

RECs Tenants Heuristics-based Agent                                   \
                    YN | Avg. Cost ($) YY | Avg. Cost ($)    Saving ($)   
10    1       3           8.393369e+06       8.185942e+06  2.074279e+05   
0     1       5           1.756083e+07       1.702578e+07  5.350472e+05   
14    1       7           2.723149e+07       2.624813e+07  9.833570e+05   
7     1      10           3.729433e+07       3.657870e+07  7.156276e+05   
18    3       3           8.347484e+06       7.896518e+06  4.509663e+05   
21    3       5           1.746958e+07       1.625629e+07  1.213298e+06   
19    3       7           2.722755e+07       2.554040e+07  1.687150e+06   
17    3      10           3.726975e+07       3.577198e+07  1.497767e+06   
5     5       3           8.366059e+06       7.623064e+06  7.429949e+05   
4     5       5           1.750588e+07       1.562452e+07  1.881361e+06   
6     5       7           2.723260e+07       2.470614e+07  2.526468e+06   
15    5      10           3.731309e+07       3.525014e+07  2.062959e+06   
16    7       3           8.375137e+06       7.390628e+06  9.845087e+05   
1     7       5           1.750280e+07       1.506245e+07  2.440345e+06   
11    7       7           2.727245e+07       2.406414e+07  3.208310e+06   
8     7      10           3.733775e+07       3.477457e+07  2.563186e+06   
22   10       3           8.388141e+06       7.109446e+06  1.278695e+06   
12   10       5           1.749439e+07       1.424986e+07  3.244529e+06   
9    10       7           2.729024e+07       2.296357e+07  4.326672e+06   
20   10      10           3.736001e+07       3.405309e+07  3.306918e+06   

                      Best Agent                                              
            % YN | Avg. Cost ($) YY | Avg. Cost ($)    Saving (%)          %  
10   2.471331       7.373864e+06       6.779690e+06  5.941739e+05   8.057837  
0    3.046822       1.550638e+07       1.430660e+07  1.199770e+06   7.737272  
14   3.611102       2.330402e+07       2.132728e+07  1.976731e+06   8.482363  
7    1.918864       3.275331e+07       3.086213e+07  1.891179e+06   5.774009  
18   5.406508       7.360440e+06       5.815452e+06  1.544988e+06  20.987584  
21   6.947141       1.554926e+07       1.355988e+07  1.989385e+06  12.797939  
19   6.196486       2.343580e+07       2.049754e+07  2.938263e+06  12.538566  
17   4.019163       3.281870e+07       3.030754e+07  2.511152e+06   7.651635  
5    8.884212       7.400265e+06       4.926165e+06  2.474100e+06  33.432159  
4   10.747360       1.557781e+07       1.271765e+07  2.860165e+06  18.363130  
6    9.276331       2.343644e+07       1.934941e+07  4.087029e+06  17.438678  
15   5.529360       3.284063e+07       2.955118e+07  3.289449e+06  10.016296  
16  11.759878       7.415951e+06       4.035733e+06  3.380218e+06  45.578091  
1   13.941589       1.559851e+07       1.194913e+07  3.649371e+06  23.396092  
11  11.762747       2.350649e+07       1.853492e+07  4.971574e+06  21.149196  
8    6.866241       3.290025e+07       2.890703e+07  3.993224e+06  12.138190  
22  15.247298       7.414950e+06       2.877395e+06  4.537554e+06  61.188396  
12  18.544859       1.557736e+07       1.080890e+07  4.768457e+06  30.612601  
9   15.853344       2.350165e+07       1.715581e+07  6.345842e+06  27.000798  
20   8.852397       3.289767e+07       2.793123e+07  4.966441e+06  15.097097

In [30]:
dirs = [item for item in os.listdir('logs/NN_NY') if "NRECS" in item]

def get_summary_df(directory):
    baseline0_rows = []
    baseline1_rows = []

    for file in os.listdir(os.path.join("logs/NN_NY", directory)):
        if file.endswith(".csv"):
            filepath = os.path.join("logs/NN_NY", directory, file)
            df_nn_ny = pd.read_csv(filepath)
            last_row = df_nn_ny.iloc[-1]
            if "BASELINE0" in file:
                baseline0_rows.append(last_row)
            elif "BASELINE1" in file:
                baseline1_rows.append(last_row)

    # Convert lists of rows to DataFrames
    if baseline0_rows:
        baseline0_df_nn_ny = pd.DataFrame(baseline0_rows)
    else:
        baseline0_df_nn_ny = pd.DataFrame()

    if baseline1_rows:
        baseline1_df_nn_ny = pd.DataFrame(baseline1_rows)
    else:
        baseline1_df_nn_ny = pd.DataFrame()

    return baseline0_df_nn_ny, baseline1_df_nn_ny

summary_data = []
for directory in dirs:
    baseline0_df_nn_ny, baseline1_df_nn_ny = get_summary_df(directory)
    n_recs = int(directory.split("_")[0].replace("NRECS", ""))
    n_tenants = int(directory.split("_")[1].replace("NTEN", ""))

    # Calculate averages for specified metrics
    result = {
        ("RECs", ""): n_recs,
        ("Tenants", ""): n_tenants,
        ("Heuristics-based Agent", "NN | Avg. Cost ($)"): baseline0_df_nn_ny["no_market_accumulated_cost"].mean() if not baseline0_df_nn_ny.empty else None,
        ("Heuristics-based Agent", "NY | Avg. Cost ($)"): baseline0_df_nn_ny["accumulated_cost"].mean() if not baseline0_df_nn_ny.empty else None,
        ("Best Agent", "NN | Avg. Cost ($)"): baseline1_df_nn_ny["no_market_accumulated_cost"].mean() if not baseline1_df_nn_ny.empty else None,
        ("Best Agent", "NY | Avg. Cost ($)"): baseline1_df_nn_ny["accumulated_cost"].mean() if not baseline1_df_nn_ny.empty else None,
    }

    summary_data.append(result)

# Create the DataFrame with MultiIndex columns
summary_df_nn_ny = pd.DataFrame(summary_data)
summary_df_nn_ny.columns = pd.MultiIndex.from_tuples(summary_df_nn_ny.columns)

summary_df_nn_ny = summary_df_nn_ny[summary_df_nn_ny["Tenants"] != 1]

summary_df_nn_ny.sort_values(by=["RECs", "Tenants"], ascending=[True, True])

RECs Tenants Heuristics-based Agent                            Best Agent  \
                    NN | Avg. Cost ($) NY | Avg. Cost ($) NN | Avg. Cost ($)   
9     1       3           8.393369e+06       8.193519e+06       7.373864e+06   
0     1       5           1.756083e+07       1.701370e+07       1.550638e+07   
12    1       7           2.723149e+07       2.619567e+07       2.330402e+07   
7     1      10           3.729433e+07       3.654305e+07       3.275331e+07   
16    3       3           8.347484e+06       8.177997e+06       7.360440e+06   
18    3       5           1.746958e+07       1.684841e+07       1.554926e+07   
17    3       7           2.722755e+07       2.630247e+07       2.343580e+07   
15    3      10           3.726975e+07       3.644551e+07       3.281870e+07   
5     5       3           8.366059e+06       8.199718e+06       7.400265e+06   
4     5       5           1.750588e+07       1.686182e+07       1.557781e+07   
6     5       7           2.723260e+07       2.631180e+07       2.343644e+07   
13    5      10           3.731309e+07       3.653416e+07       3.284063e+07   
14    7       3           8.375137e+06       8.190777e+06       7.415951e+06   
1     7       5           1.750280e+07       1.683140e+07       1.559851e+07   
10    7       7           2.727245e+07       2.632422e+07       2.350649e+07   
8     7      10           3.733775e+07       3.659687e+07       3.290025e+07   

                       
   NY | Avg. Cost ($)  
9        7.187492e+06  
0        1.489157e+07  
12       2.246997e+07  
7        3.202294e+07  
16       7.214876e+06  
18       1.499968e+07  
17       2.263801e+07  
15       3.210535e+07  
5        7.253063e+06  
4        1.500511e+07  
6        2.258780e+07  
13       3.214082e+07  
14       7.266054e+06  
1        1.501294e+07  
10       2.264171e+07  
8        3.218355e+07

In [31]:
merged = pd.merge(summary_df_yn_yy, summary_df_nn_ny, on=["RECs", "Tenants"])

merged = merged[[
    ("RECs", ""),
    ("Tenants", ""),
    ("Heuristics-based Agent", "NN | Avg. Cost ($)"),
    ("Heuristics-based Agent", "NY | Avg. Cost ($)"),
    ("Heuristics-based Agent", "YY | Avg. Cost ($)"),
    ("Heuristics-based Agent", "Saving ($)"),
    ("Heuristics-based Agent", "%"),
    ("Best Agent", "NN | Avg. Cost ($)"),
    ("Best Agent", "NY | Avg. Cost ($)"),
    ("Best Agent", "YY | Avg. Cost ($)"),
    ("Best Agent", "Saving (%)"),
    ("Best Agent", "%"),
]]

merged.loc[merged[("RECs", "")] == 1, ("Heuristics-based Agent", "NY | Avg. Cost ($)")] = None
merged.loc[merged[("RECs", "")] == 1, ("Best Agent", "NY | Avg. Cost ($)")] = None

merged = merged.sort_values(by=["RECs", "Tenants"], ascending=[True, True]).reset_index(drop=True)

merged[merged["RECs"] != 10]

RECs Tenants Heuristics-based Agent                                        \
                    NN | Avg. Cost ($) NY | Avg. Cost ($) YY | Avg. Cost ($)   
0     1       3           8.393369e+06                NaN       8.185942e+06   
1     1       5           1.756083e+07                NaN       1.702578e+07   
2     1       7           2.723149e+07                NaN       2.624813e+07   
3     1      10           3.729433e+07                NaN       3.657870e+07   
4     3       3           8.347484e+06       8.177997e+06       7.896518e+06   
5     3       5           1.746958e+07       1.684841e+07       1.625629e+07   
6     3       7           2.722755e+07       2.630247e+07       2.554040e+07   
7     3      10           3.726975e+07       3.644551e+07       3.577198e+07   
8     5       3           8.366059e+06       8.199718e+06       7.623064e+06   
9     5       5           1.750588e+07       1.686182e+07       1.562452e+07   
10    5       7           2.723260e+07       2.631180e+07       2.470614e+07   
11    5      10           3.731309e+07       3.653416e+07       3.525014e+07   
12    7       3           8.375137e+06       8.190777e+06       7.390628e+06   
13    7       5           1.750280e+07       1.683140e+07       1.506245e+07   
14    7       7           2.727245e+07       2.632422e+07       2.406414e+07   
15    7      10           3.733775e+07       3.659687e+07       3.477457e+07   

                                    Best Agent                     \
      Saving ($)          % NN | Avg. Cost ($) NY | Avg. Cost ($)   
0   2.074279e+05   2.471331       7.373864e+06                NaN   
1   5.350472e+05   3.046822       1.550638e+07                NaN   
2   9.833570e+05   3.611102       2.330402e+07                NaN   
3   7.156276e+05   1.918864       3.275331e+07                NaN   
4   4.509663e+05   5.406508       7.360440e+06       7.214876e+06   
5   1.213298e+06   6.947141       1.554926e+07       1.499968e+07   
6   1.687150e+06   6.196486       2.343580e+07       2.263801e+07   
7   1.497767e+06   4.019163       3.281870e+07       3.210535e+07   
8   7.429949e+05   8.884212       7.400265e+06       7.253063e+06   
9   1.881361e+06  10.747360       1.557781e+07       1.500511e+07   
10  2.526468e+06   9.276331       2.343644e+07       2.258780e+07   
11  2.062959e+06   5.529360       3.284063e+07       3.214082e+07   
12  9.845087e+05  11.759878       7.415951e+06       7.266054e+06   
13  2.440345e+06  13.941589       1.559851e+07       1.501294e+07   
14  3.208310e+06  11.762747       2.350649e+07       2.264171e+07   
15  2.563186e+06   6.866241       3.290025e+07       3.218355e+07   

                                                
   YY | Avg. Cost ($)    Saving (%)          %  
0        6.779690e+06  5.941739e+05   8.057837  
1        1.430660e+07  1.199770e+06   7.737272  
2        2.132728e+07  1.976731e+06   8.482363  
3        3.086213e+07  1.891179e+06   5.774009  
4        5.815452e+06  1.544988e+06  20.987584  
5        1.355988e+07  1.989385e+06  12.797939  
6        2.049754e+07  2.938263e+06  12.538566  
7        3.030754e+07  2.511152e+06   7.651635  
8        4.926165e+06  2.474100e+06  33.432159  
9        1.271765e+07  2.860165e+06  18.363130  
10       1.934941e+07  4.087029e+06  17.438678  
11       2.955118e+07  3.289449e+06  10.016296  
12       4.035733e+06  3.380218e+06  45.578091  
13       1.194913e+07  3.649371e+06  23.396092  
14       1.853492e+07  4.971574e+06  21.149196  
15       2.890703e+07  3.993224e+06  12.138190

In [32]:
# format to 3 decimal places
def format_float(value):
    if pd.isna(value):
        return value
    
    # if lower than 0.001, use scientific notation
    elif value > 10**2:
        return f"{value:.3e}"
    else:
        return f"{value:.2f}"

# Apply the formatting function to the DataFrame
formatted_df = merged.applymap(format_float)

# hide index
formatted_df = formatted_df.drop([("RECs", "",), ("Tenants", "")], axis=1)
formatted_df.insert(0, ("RECs", ""), merged[("RECs", "")])
formatted_df.insert(1, ("Tenants", ""), merged[("Tenants", "")])

formatted_df.style.hide(axis="index")

In [33]:
df = merged.copy()
df.columns = ['RECs', 'Tenants'] + ['_'.join(col).strip() for col in df.columns[2:]]
df = df[['RECs', 'Tenants', 'Best Agent_%']]

In [34]:
# Pivot the data to make a grid (RECs vs Tenants)
pivot_df = df.pivot(index='Tenants', columns='RECs', values='Best Agent_%')

# Create interpolated heatmap
fig = go.Figure(data=go.Heatmap(
    z=pivot_df.values,
    x=pivot_df.columns,        # RECs
    y=pivot_df.index,          # Tenants
    colorscale='Viridis',
    colorbar=dict(title='Savings (%)'),
    # zsmooth='best',             # Interpolation,
))

fig.update_xaxes(
    tickvals=[1, 3, 5, 7],
    ticktext=["1", "3", "5", "7"],  # Set the tick labels
    range=[0, 8]  # Set the x-axis range
)

fig.update_yaxes(
    tickvals=[3, 5, 7, 10],
    ticktext=["3", "5", "7", "10"],  # Set the tick labels
    range=[2, 11]  # Set the y-axis range
)

fig.update_layout(
    xaxis_title='Number of RECs',
    yaxis_title='Number of Tenants',
    height=500,
    width=700,
    margin=dict(l=20, r=20, t=20, b=20)
)

fig.show()

In [35]:
# by tenant
tenants = df["Tenants"].unique()
fig = go.Figure()

for n_tenants in tenants:    

    base = df[df["Tenants"] == n_tenants].copy()

    x = base["RECs"].values.reshape(-1, 1)
    y = base["Best Agent_%"].values

    # Fit linear regression
    model = LinearRegression()
    model.fit(x, y)

    y_pred = model.predict(x)
    error = model.score(x, y)

    print(f"Tenants: {n_tenants} | y = {model.coef_[0]:.4f} * x + {model.intercept_:.4f} | r² = {error:.4f}")

    # Get slope and intercept
    slope = model.coef_[0]
    intercept = model.intercept_


    fig.add_trace(go.Scatter(
        x=x.flatten(),
        y=y,
        mode='markers',
        marker=dict(color='blue', size=8),
        showlegend=False,
    ))

    fig.add_trace(go.Scatter(
        x=x.flatten(),
        y=model.predict(x),
        mode='lines',
        name=f"Tenants = {n_tenants}<br>y = {slope:.4f} * x + {intercept:.4f}<br>"
    ))

fig.update_layout(
    xaxis_title='Number of RECs',
    yaxis_title='Savings (%)',
    height=500,
    width=800,
    margin=dict(l=20, r=20, t=20, b=20),
    # showlegend=False,
)

write_image(fig, "linear_regression.png", show=True)
write_image(fig, "linear_regression.pdf")

Tenants: 3 | y = 6.2503 * x + 2.0129 | r² = 0.9998
Tenants: 5 | y = 2.6271 * x + 5.0653 | r² = 0.9996
Tenants: 7 | y = 2.1450 * x + 6.3221 | r² = 0.9974
Tenants: 10 | y = 1.0729 * x + 4.6036 | r² = 0.9982


In [10]:
num_recs = 5
num_tenants = 5

dir = f"NRECS{num_recs}_NTEN{num_tenants}"
baseline1_rows = []
baseline0_rows = []

for file in os.listdir(os.path.join("logs/YN_YY", dir)):
    if file.endswith(".csv"):
        filepath = os.path.join("logs/YN_YY", dir, file)
        
        if "transactions" in filepath:
            continue
        
        print(filepath)
        df_yn_yy = pd.read_csv(filepath)
        df_yn_yy["REC"] = int(file.split("_")[-1].replace(".csv", ""))
        rows = df_yn_yy.to_dict(orient="records")

        if "BASELINE1" in file:
            baseline1_rows += rows

        if "BASELINE0" in file:
            baseline0_rows += rows

simulation = pd.DataFrame(baseline1_rows).sort_values(by=["REC", "step"], ascending=[True, True]).reset_index(drop=True)
simulation_baseline = pd.DataFrame(baseline0_rows).sort_values(by=["REC", "step"], ascending=[True, True]).reset_index(drop=True)
# transactions = pd.read_csv(os.path.join("logs/YN_YY", dir, "transactions.csv"))

logs/YN_YY/NRECS5_NTEN5/NRECS5_NTEN5_BASELINE1rec_3.csv
logs/YN_YY/NRECS5_NTEN5/NRECS5_NTEN5_BASELINE0rec_0.csv
logs/YN_YY/NRECS5_NTEN5/NRECS5_NTEN5_BASELINE1rec_1.csv
logs/YN_YY/NRECS5_NTEN5/NRECS5_NTEN5_BASELINE0rec_4.csv
logs/YN_YY/NRECS5_NTEN5/NRECS5_NTEN5_BASELINE0rec_1.csv
logs/YN_YY/NRECS5_NTEN5/NRECS5_NTEN5_BASELINE1rec_2.csv
logs/YN_YY/NRECS5_NTEN5/NRECS5_NTEN5_BASELINE0rec_2.csv
logs/YN_YY/NRECS5_NTEN5/NRECS5_NTEN5_BASELINE0rec_3.csv
logs/YN_YY/NRECS5_NTEN5/NRECS5_NTEN5_BASELINE1rec_0.csv
logs/YN_YY/NRECS5_NTEN5/NRECS5_NTEN5_BASELINE1rec_4.csv


In [11]:
num_recs = 5
num_tenants = 5

dir = f"NRECS{num_recs}_NTEN{num_tenants}"
baseline1_rows = []
baseline0_rows = []

for file in os.listdir(os.path.join("logs/NN_NY", dir)):
    if file.endswith(".csv"):
        filepath = os.path.join("logs/NN_NY", dir, file)
        
        if "transactions" in filepath:
            continue
        
        print(filepath)
        df_nn_ny = pd.read_csv(filepath)
        df_nn_ny["REC"] = int(file.split("_")[-1].replace(".csv", ""))
        rows = df_nn_ny.to_dict(orient="records")

        if "BASELINE1" in file:
            baseline1_rows += rows

        if "BASELINE0" in file:
            baseline0_rows += rows

simulation2 = pd.DataFrame(baseline1_rows).sort_values(by=["REC", "step"], ascending=[True, True]).reset_index(drop=True)
simulation2_baseline = pd.DataFrame(baseline0_rows).sort_values(by=["REC", "step"], ascending=[True, True]).reset_index(drop=True)
# transactions = pd.read_csv(os.path.join("logs/NN_NY", dir, "transactions.csv"))

logs/NN_NY/NRECS5_NTEN5/NRECS5_NTEN5_BASELINE1rec_3.csv
logs/NN_NY/NRECS5_NTEN5/NRECS5_NTEN5_BASELINE0rec_0.csv
logs/NN_NY/NRECS5_NTEN5/NRECS5_NTEN5_BASELINE1rec_1.csv
logs/NN_NY/NRECS5_NTEN5/NRECS5_NTEN5_BASELINE0rec_4.csv
logs/NN_NY/NRECS5_NTEN5/NRECS5_NTEN5_BASELINE0rec_1.csv
logs/NN_NY/NRECS5_NTEN5/NRECS5_NTEN5_BASELINE1rec_2.csv
logs/NN_NY/NRECS5_NTEN5/NRECS5_NTEN5_BASELINE0rec_2.csv
logs/NN_NY/NRECS5_NTEN5/NRECS5_NTEN5_BASELINE0rec_3.csv
logs/NN_NY/NRECS5_NTEN5/NRECS5_NTEN5_BASELINE1rec_0.csv
logs/NN_NY/NRECS5_NTEN5/NRECS5_NTEN5_BASELINE1rec_4.csv


In [12]:
simulation.columns

Index(['step', 'total_demand', 'energy_pool', 'cost', 'tenant_0_cost',
       'tenant_1_cost', 'tenant_2_cost', 'tenant_3_cost', 'tenant_4_cost',
       'sum_tenant_costs', 'accumulated_cost', 'no_market_accumulated_cost',
       'absolute_accumulated_saving', 'percentage_accumulated_saving',
       'tenant_imports', 'tenant_exports', 'sim_nrecs', 'sim_ntenants', 'REC'],
      dtype='object')

In [13]:
energy_price = pd.read_csv("price.csv").iloc[:100]

In [14]:
fig = go.Figure()
colors = ["#E57373", "#FFB74D", "#81C784", "#4FC3F7", "#BA68C8", "#64B5F6", "#F06292"]

for rec in range(num_recs):
    df_yn_yy = simulation[simulation["REC"] == rec].iloc[:100]
    fig.add_trace(
        go.Scatter(
            x=df_yn_yy["step"],
            y=df_yn_yy["accumulated_cost"],
            name=f"REC {rec}",
            line=dict(color=colors[rec % len(colors)]),
            mode="lines",
            showlegend=True,
        )
    )

fig.update_layout(
    height=500,
    width=700,
    xaxis_title="Step",
    yaxis_title="Avg. Trading Costs ($)",
    margin=dict(l=20, r=20, t=20, b=20),
    yaxis=dict(
        range=[0, 250000],
    ),
)

write_image(fig, "trading_costs_study.pdf", show=True)
write_image(fig, "trading_costs_study.png")

In [15]:
# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

for rec in range(num_recs):
    df_yn_yy = simulation[simulation["REC"] == rec].iloc[:100]
    fig.add_trace(
        go.Scatter(
            x=df_yn_yy["step"],
            y=df_yn_yy["accumulated_cost"],
            name=f"REC {rec}",
            line=dict(color=colors[rec % len(colors)]),
            mode="lines",
            showlegend=True,
        ),
        secondary_y=False,
    )

# Add energy price trace
fig.add_trace(
    go.Scatter(
        x=df_yn_yy["step"],
        y=energy_price["PRICE"],
        name="Energy Price",
        line=dict(color="black"),
        mode="lines",
        showlegend=True,
    ),
    secondary_y=True,
)

fig.update_layout(
    height=500,
    width=700,
    xaxis_title="Step",
    margin=dict(l=20, r=20, t=20, b=20),
)

fig.update_yaxes(title_text="Avg. Trading Costs ($)", range=[0, 250000], secondary_y=False, showgrid=True)
fig.update_yaxes(title_text="€/MWh", range=[0, 70], secondary_y=True, showgrid=False)

fig.update_xaxes(
    tickvals=[0, 10, 30, 50, 70, 90],
    ticktext=["0", "10", "30", "50", "70", "90"],
    showgrid=True,
)

write_image(fig, "trading_costs_and_energy_price_study.pdf", show=True)
write_image(fig, "trading_costs_and_energy_price_study.png")

In [16]:
fig = go.Figure()

df_yn_yy = simulation.drop("REC", axis=1).groupby("step").mean()
df_nn_ny = simulation2.drop("REC", axis=1).groupby("step").mean()

df_yn_yy_baseline = simulation_baseline.drop("REC", axis=1).groupby("step").mean()
df_nn_ny_baseline = simulation2_baseline.drop("REC", axis=1).groupby("step").mean()

# fig.add_trace(
#     go.Scatter(
#         x=df_yn_yy_baseline.index,
#         y=df_yn_yy_baseline["no_market_accumulated_cost"],
#         name=f"Heuristic | No trading",
#         line=dict(color='orange'),
#         mode="lines",
#         showlegend=True,
#     )
# )

# fig.add_trace(
#     go.Scatter(
#         x=df_nn_ny_baseline.index,
#         y=df_nn_ny_baseline["accumulated_cost"],
#         name=f"Heuristic | Inter-REC trading",
#         line=dict(color='brown'),
#         mode="lines",
#         showlegend=True,
#     )
# )

# fig.add_trace(
#     go.Scatter(
#         x=df_yn_yy_baseline.index,
#         y=df_yn_yy_baseline["accumulated_cost"],
#         name=f"Heuristic | Inter-REC trading<br>+ Intra-REC exchange",
#         line=dict(color='purple'),
#         mode="lines",
#         showlegend=True,
#     )
# )

fig.add_trace(
    go.Scatter(
        x=df_yn_yy.index,
        y=df_yn_yy["no_market_accumulated_cost"],
        name=f"No trading",
        line=dict(color='red'),
        mode="lines",
        showlegend=True,
    )
)

fig.add_trace(
    go.Scatter(
        x=df_nn_ny.index,
        y=df_nn_ny["accumulated_cost"],
        name=f"Only Inter-REC trading",
        line=dict(color='green'),
        mode="lines",
        showlegend=True,
    )
)

fig.add_trace(
    go.Scatter(
        x=df_yn_yy.index,
        y=df_yn_yy["accumulated_cost"],
        name=f"Inter-REC trading and<br>Intra-REC exchange",
        line=dict(color='blue'),
        mode="lines",
        showlegend=True,
    )
)

fig.update_layout(
    height=500,
    width=800,
    xaxis_title="Step",
    yaxis_title="Trading Costs ($)",
    margin=dict(l=20, r=20, t=20, b=20),
)

write_image(fig, "trading_costs_compared.pdf", show=True)
write_image(fig, "trading_costs_compared.png")